In [4]:
import scrapy
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from scrapy.selector import Selector
import time
import pandas as pd
from datetime import datetime
from datetime import timedelta


search_str = 'cybersecurity'
last_x_weeeks = 4

### 1. Search terms defined abvove

In [5]:
search_str = search_str.replace(' ','+')
name = 'nytimes'
allowed_domains = ['www.nytimes.com']
start_urls = ['https://www.nytimes.com']


# set up the driver
chrome_options = Options()
# chrome_options.add_argument("--headless") # uncomment if don't want to appreciate the sight of a possessed browser
driver = webdriver.Chrome(chrome_options)
driver.get(F"https://www.nytimes.com/search?query={search_str}")


time.sleep(2)
date_range_btn = driver.find_element(by = By.XPATH,value = "//*[@id='site-content']/div/div[1]/div[2]/div/div/div[1]/div/div/button/label")
date_range_btn.click()

time.sleep(2)
past_year_btn = driver.find_element(by = By.XPATH,value = "//*[@id='site-content']/div/div[1]/div[2]/div/div/div[1]/div/div/div/ul/li[5]/button")
past_year_btn.click()


time.sleep(2)
date_range_btn = driver.find_element(by = By.XPATH,value = "//*[@id='site-content']/div/div[1]/div[1]/form/div[2]/div/select")
date_range_btn.click()



time.sleep(2)
past_year_btn = driver.find_element(by = By.XPATH,value = "//*[@id='site-content']/div/div[1]/div[1]/form/div[2]/div/select/option[2]")
past_year_btn.click()

try:
    time.sleep(10)
    next_btn = driver.find_element(by = By.XPATH,value = "//*[@id='site-content']/div/div[2]/div[2]/div/button")
    next_btn.click()
    indice = 1
except Exception as e:
    try:
        time.sleep(10)
        next_btn = driver.find_element(by = By.XPATH,value = "//*[@id='site-content']/div/div[2]/div[3]/div/button")
        next_btn.click()
        indice = 2
    except Exception as e1:
        print(e1)


time.sleep(2)

In [6]:
html = [driver.page_source]

### 2. Define function to scrap each news page

In [8]:
def get_pages(html,index):
    rows = []
    for page in html:
        resp = Selector(text = page)
        #print(resp)
        results = resp.xpath(f"//*[@id='site-content']/div/div[2]/div[{index}]/ol") # result iterator
        
        print(results)    
        dataUriList =results.xpath(f"//*[@id='site-content']/div/div[2]/div[{index}]/ol/li[*]")
        print(dataUriList)
        row = []
        for i,item in enumerate(dataUriList):

            try:

                #title
                title = item.xpath(f"//*[@id='site-content']/div/div[2]/div[{index}]/ol/li[{i+1}]/div/div/div/a/h4/text()").get()
                print(title)

            except:

                title = 'No Title'
            

            try:

                #date
                date_div = item.xpath(f"//*[@id='site-content']/div/div[2]/div[{index}]/ol/li[{i+1}]/div/div/div/a/span/text()[2]").get()
                a_list = date_div.split(',')
                date = f"{a_list[0]},{a_list[1]}"
                print(date)
                datetime_parsed =datetime.strptime(date, "%B %d, %Y")
                print(datetime_parsed)
            
            except:

                datetime_parsed = datetime.strptime("January 01, 2000", "%B %d, %Y")


            try:

                link = item.xpath(f"//*[@id='site-content']/div/div[2]/div[{index}]/ol/li[{i+1}]/div/div/div/a/@href").get()
                #print(link)

            except:
                link = 'No link'


            try:

                description = item.xpath(f"//*[@id='site-content']/div/div[2]/div[{index}]/ol/li[{i+1}]/div/div/div/a/p[1]/text()").get()
                #description = description_div.xpath(".//text()").get().strip()
                #print(description)
            
            except:
                description = 'No description'

            
            try:

                author = item.xpath(f"//*[@id='site-content']/div/div[2]/div[{index}]/ol/li[{i+1}]/div/div/div/a/p[2]/text()").get()
            except:
                author = 'No author'

            row = [title,datetime_parsed,description,link,author]
            rows.append(row)

        return rows

### 3. Poll each page

In [9]:
headers = ['Title','Publish Date','Description','Url','Author']
news = get_pages(html,indice)

[<Selector query="//*[@id='site-content']/div/div[2]/div[2]/ol" data='<ol data-testid="search-results" aria...'>]
[<Selector query="//*[@id='site-content']/div/div[2]/div[2]/ol/li[*]" data='<li class="css-1l4w6pd" data-testid="...'>, <Selector query="//*[@id='site-content']/div/div[2]/div[2]/ol/li[*]" data='<li class="css-1l4w6pd" data-testid="...'>, <Selector query="//*[@id='site-content']/div/div[2]/div[2]/ol/li[*]" data='<li class="css-1l4w6pd" data-testid="...'>, <Selector query="//*[@id='site-content']/div/div[2]/div[2]/ol/li[*]" data='<li class="css-1l4w6pd" data-testid="...'>, <Selector query="//*[@id='site-content']/div/div[2]/div[2]/ol/li[*]" data='<li class="css-1yvwzdo"><div class="c...'>, <Selector query="//*[@id='site-content']/div/div[2]/div[2]/ol/li[*]" data='<li class="css-1l4w6pd" data-testid="...'>, <Selector query="//*[@id='site-content']/div/div[2]/div[2]/ol/li[*]" data='<li class="css-1l4w6pd" data-testid="...'>, <Selector query="//*[@id='site-content']/div/div[2]/

### 4. Order and filter news by date

In [10]:
filter_date = datetime.today()-timedelta(weeks=last_x_weeeks)


news_df = pd.DataFrame(data=news,columns=headers)
news_df_sorted = news_df.sort_values(by= 'Publish Date', ascending=False)
news_df_filtered = news_df_sorted[news_df_sorted['Publish Date']>=filter_date]

### 5. Build the output file to be used as chat gpt input

In [11]:
 
print(news_df_filtered)
news_df_filtered.to_csv('OutputTestFiles/nyt_test.csv',sep=";")

                                                Title Publish Date  \
0            Where Have All the Chinese I.P.O.s Gone?   2024-06-26   
2   What the Arrival of A.I. Phones and Computers ...   2024-06-26   
3   Cyberattacks Disrupt Car Sales by Dealers in U...   2024-06-22   
5   Can A.I. Answer the Needs of Smaller Businesse...   2024-06-18   
6   Lawmakers Question Microsoft’s President About...   2024-06-14   
7   The Aftermath of a U.K. Cyberattack: Blood Sho...   2024-06-14   
9   A New Measure Shows C.E.O. Pay at Even More As...   2024-06-09   
12  A Republican Election Clerk vs. Trump Die-Hard...   2024-06-09   
11  ‘This Is Going to Be Painful’: How a Bold A.I....   2024-06-07   
14  How Top N.Y. Officials Helped a Lobbyist Cash ...   2024-06-06   

                                          Description  \
0   Chinese companies’ stock market listings once ...   
2   Apple, Microsoft and Google need more access t...   
3   The attacks on a software provider, CDK Global...   
5